# importing the library

In [25]:
import pandas as pd
import gensim as gs
import warnings
warnings.filterwarnings("ignore")
import nltk
import string
from nltk.corpus import wordnet
import re
from nltk import pos_tag
from contractions import CONTRACTION_MAP
from gensim.models import Word2Vec

# Count that each theme has on which model is made


In [2]:
df = pd.read_csv('/home/techm/Downloads/Qaiser/sentiment_classification/olm_data_theme_comment.csv')
df['Theme (Issues)'] = df['Theme (Issues)'].map(lambda x: x.lower())
print(df['Theme (Issues)'].value_counts())
print(df.shape)

df.reset_index(drop=True, inplace=True)
df = df[['Comments']]



chat issue                             1368
unable to find content                  400
performance issues                      397
unable to order                         363
non functional website                  320
unable to make payment                  313
customer demands                        269
looping                                 170
vague comment                           161
unable to enter/update address          156
navigation:unable to proceed            148
navigation: links                       147
blank page                              145
chat issue                               91
unable to activate                       85
unable to login                          85
logout                                   79
zipcode issue                            74
unable ot login                          72
unable to add to cart                    58
unable to receive verification code      52
unable to update address                 48
uverse session                  

# Using Data of different lob
   
   To derive semantic relationship among word, model need to be traind on more data, though the data are from different      
   lob the word will still have same meaning   
   
    

In [3]:



# Data_CWS  contain 1675 comments
df1  = pd.read_excel("/home/techm/Downloads/Qaiser/sentiment_classification/DATA_CWS.xlsx")
df1 = df1[['Comments']]
df1.dropna(inplace=True)
print("Shape of DATA_CWS",df1.shape)
df1.reset_index(drop=True, inplace=True)
#
df2 = pd.read_csv("/home/techm/Downloads/Qaiser/sentiment_classification/OpinionLabCWSdata012017_072018.csv")
df2 = df2[['Comments']]
df2 = df2[:12130]
df2.dropna(inplace=True)
print("shape of OpinionLabCWSdata012017_072018.csv", df2.shape)
df2.reset_index(drop = True, inplace=True)
#
df3 = pd.read_excel('/home/techm/Downloads/Qaiser/sentiment_classification/OpinionLabServicesData012018_062018.xlsx')
df3 = df3[['Comments']]
df3 = df3[:19032]
df3.dropna(inplace=True)
print("shape of OpinionLabServicesData012018_062018", df3.shape)
df3.reset_index(drop = True, inplace=True)



Shape of DATA_CWS (1337, 1)
shape of OpinionLabCWSdata012017_072018.csv (8141, 1)
shape of OpinionLabServicesData012018_062018 (19032, 1)


# concatenating pandas dataframe


In [4]:
concat_df = pd.concat([df,df1,df2,df3],axis=0)
concat_df.dropna(inplace=True)
print("concat_df.shape",concat_df.shape)
concat_df

concat_df.shape (33691, 1)


,Comments
0,Have option for a one time pay by others on a ...
1,Rep disconnected chat
2,Rep disconnected chat improp�
3,Chat reps don’t know what they’re doing
4,Apparently your website is very slow and hard ...
5,Your website is slow and cumbersome!
6,Currently looking at phones for upgrade. Would...
7,Continue doesn't work when entering billing ad...
8,Make it handicapped friendly I am legally blind
9,Page isn't loading.


# Text   preprocessing start

In [41]:
def tokenize_text(text):
    '''
    function used for tokenizing string
    '''
    
    tokenize_word = [nltk.word_tokenize(sentence) for sentence in text]
    return tokenize_word





def expand_contractions(sentence, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                     flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence
     
    
    


def remove_repeated_characters(tokens):
    '''
    function where repeated character  are in token get replaced, using wordnet dictionary
    
    
    example
    sample_token = "'My, 'schooool', 'is', 'realllllyyy', 'amaaazingggg'"
    output_token = 'My', 'school', 'is', 'really', 'amazing'"
    '''
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word !=old_word else new_word
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens
    

def remove_characters_before_tokenization(sentence,keep_apostrophes=False):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9 ]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence


# Text Preprocessing Step explained
   
   1. First step involve is to remove special character which include punctuation, special character, keeping apostrophes
      by keeping apostrophes true
      
   2. Second step used dictionary file contraction mapping which will expand word like "aren't": "are not",
      "can't": "cannot"
   
   3. Third step is to tokenized sentence
   
   4. After the token is created, we  remove repeated character.
   
      for example 
      sample_token = "'My, 'schooool', 'is', 'realllllyyy', 'amaaazingggg'", 
      
      output_token = 'My', 'school', 'is', 'really', 'amazing'" 
      Using wordnet dictionary
   

In [42]:
text = [x for x in concat_df.Comments]

In [43]:
expanded_corpus = [expand_contractions(sentence, CONTRACTION_MAP) for sentence in text] 

In [44]:
cleaned_corpus= [remove_characters_before_tokenization(sentence,keep_apostrophes=False) for sentence in expanded_corpus] 

In [45]:
cleaned_corpus

['Have option for a one time pay by others on a family plan without disturbing the plan owners card information allow one time payment card information to be deleted  Person on chat wanted money upfront when ordering a phone when the site said no charges until next bill',
 'Rep disconnected chat',
 'Rep disconnected chat improp',
 'Chat reps dont know what theyre doing',
 'Apparently your website is very slow and hard to navigate on an ipad and chromebook',
 'Your website is slow and cumbersome',
 'Currently looking at phones for upgrade Would prefer a side bar for specifying and narrowing search rather than only having the Show More button Would also suggest smaller tiles so for standard Zoom percentage so comparison between products is easier',
 'Continue does not work when entering billing address',
 'Make it handicapped friendly I am legally blind',
 'Page is not loading',
 'Its impossible to put a down payment on the phone  When you update the cart it shows two phones One with new

In [48]:
tokenized_word1 = [nltk.word_tokenize(x) for x in cleaned_corpus]

In [49]:
tokenized_word1

[['Have',
  'option',
  'for',
  'a',
  'one',
  'time',
  'pay',
  'by',
  'others',
  'on',
  'a',
  'family',
  'plan',
  'without',
  'disturbing',
  'the',
  'plan',
  'owners',
  'card',
  'information',
  'allow',
  'one',
  'time',
  'payment',
  'card',
  'information',
  'to',
  'be',
  'deleted',
  'Person',
  'on',
  'chat',
  'wanted',
  'money',
  'upfront',
  'when',
  'ordering',
  'a',
  'phone',
  'when',
  'the',
  'site',
  'said',
  'no',
  'charges',
  'until',
  'next',
  'bill'],
 ['Rep', 'disconnected', 'chat'],
 ['Rep', 'disconnected', 'chat', 'improp'],
 ['Chat', 'reps', 'dont', 'know', 'what', 'theyre', 'doing'],
 ['Apparently',
  'your',
  'website',
  'is',
  'very',
  'slow',
  'and',
  'hard',
  'to',
  'navigate',
  'on',
  'an',
  'ipad',
  'and',
  'chromebook'],
 ['Your', 'website', 'is', 'slow', 'and', 'cumbersome'],
 ['Currently',
  'looking',
  'at',
  'phones',
  'for',
  'upgrade',
  'Would',
  'prefer',
  'a',
  'side',
  'bar',
  'for',
  'spe

In [50]:
spelling_corrected_token = []

for i in range(len(tokenized_word1)):
    cleaned_spelling =remove_repeated_characters(tokenized_word1[i])
    spelling_corrected_token.append(cleaned_spelling)



In [51]:
len(spelling_corrected_token)

33691

In [52]:
#converting All the uppercase token to lower case
lower_token = []

for i in range(len(spelling_corrected_token)):
    lower_token1 = [token.lower() for token in spelling_corrected_token[i]]
    lower_token.append(lower_token1)
    

In [53]:
lower_token

[['have',
  'option',
  'for',
  'a',
  'one',
  'time',
  'pay',
  'by',
  'others',
  'on',
  'a',
  'family',
  'plan',
  'without',
  'disturbing',
  'the',
  'plan',
  'owners',
  'card',
  'information',
  'allow',
  'one',
  'time',
  'payment',
  'card',
  'information',
  'to',
  'be',
  'deleted',
  'person',
  'on',
  'chat',
  'wanted',
  'money',
  'upfront',
  'when',
  'ordering',
  'a',
  'phone',
  'when',
  'the',
  'site',
  'said',
  'no',
  'charges',
  'until',
  'next',
  'bill'],
 ['rep', 'disconnected', 'chat'],
 ['rep', 'disconnected', 'chat', 'improp'],
 ['chat', 'reps', 'dont', 'know', 'what', 'theyre', 'doing'],
 ['apparently',
  'your',
  'website',
  'is',
  'very',
  'slow',
  'and',
  'hard',
  'to',
  'navigate',
  'on',
  'an',
  'ipad',
  'and',
  'chromebok'],
 ['your', 'website', 'is', 'slow', 'and', 'cumbersome'],
 ['currently',
  'looking',
  'at',
  'phones',
  'for',
  'upgrade',
  'would',
  'prefer',
  'a',
  'side',
  'bar',
  'for',
  'spec

# building word2vec Algorithm of cleaned corrected word using Gensim in 50 dimension

In [54]:
# to check unique word in our vocabulary
all_words = []
for i in range(len(lower_token)):
    for word in lower_token[i]:
        all_words.append(word)
print("Unique token which is present in our vocab",len(set(all_words)))

Unique token which is present in our vocab 19328


In [59]:
model = Word2Vec(spelling_corrected_token, size=50, window=5, min_count=2, workers=2)

In [60]:
model

In [61]:
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}
w2v

{'to': array([ 2.06299639, -2.96542549, -0.11080936, -1.16890788, -1.57442534,
         0.7315768 , -0.05094594,  0.46343878,  3.348804  , -1.19695055,
        -1.8577491 ,  2.13233113,  1.0102824 ,  0.44228223,  0.20279515,
        -0.35460719,  1.90886974, -0.45204547, -1.66751122, -0.44063315,
        -1.54824471,  0.73830038,  1.0457027 ,  3.02496743, -1.44326484,
        -0.07182782,  0.05766496,  2.17748594, -2.40665174, -2.1143055 ,
        -1.92751741, -0.6126833 , -0.60528278, -0.79259712,  1.16979253,
        -0.44413298,  0.19910583,  0.48250231, -1.0713166 ,  0.08048327,
         0.06650425,  0.55068302, -1.5882951 ,  0.45067886, -0.57879251,
        -1.08641469, -0.26203158,  0.45360422,  0.19264539,  0.12977038], dtype=float32),
 'I': array([-1.04819679, -1.24272609,  1.27936935, -1.82387996, -1.91763151,
         1.38816404, -1.40147626, -1.08000791,  2.0834074 ,  1.85461664,
        -1.32550609,  1.03133166,  2.03029966, -0.86116034, -1.13885033,
         0.54701632,  0

In [58]:
model.save("word2vec_50.model")
